In [1]:
#导入所需模块
import numpy as np
import os
import cv2

# 导入训练图片

1、读取图片

In [2]:
# 返回的是480张100*100的矩阵，和index
listdir = os.listdir('./face_train')
names = [d for d in listdir if not d.startswith('.')]
images = []
target = []
for index,dir in enumerate(names):
    for i in range(1, 11):
        gray = cv2.imread('./face_train/%s/%d.jpg' % (dir, i))  # 三维图片
        
        #图像均衡化
        gray_equalized = histogram_equalization(gray)
        
        gray_ = gray_equalized[:, :, 0]  # 二维数组
        gray_ = cv2.resize(gray_, dsize=(100, 100))
        images.append(gray_)
        target.append(index)
images = np.asarray(images)
target = np.asarray(target)

2、转换成矩阵

In [3]:
#图像数据转换特征矩阵
image_data = []

for image in images:
    #转换成一维的数组
    data = image.flatten()
    image_data.append(data)

#print(image_data)

In [4]:
#转换为numpy数组
X_train = np.array(image_data)
y_train = target
print(type(X_train))
print(X_train.shape)

<class 'numpy.ndarray'>
(200, 10000)


# 导入测试向量

In [5]:
listdir_test = os.listdir('./face_test')
names_test = [d for d in listdir_test if not d.startswith('.')]
print(names_test)
images_test = []
target_test = []
for index_test,dir_test in enumerate(names_test):
    for i in range(1, 31):
        gray_test = cv2.imread('./face_test/%s/%d.jpg' % (dir_test, i))  # 三维图片
        
        #图像均衡化
        gray_test_equalized = histogram_equalization(gray_test)
        
        gray_test_ = gray_test_equalized[:, :, 0]  # 二维数组
        gray_test_ = cv2.resize(gray_test_, dsize=(100, 100))
        images_test.append(gray_test_)
        target_test.append(index_test)
images_test = np.asarray(images_test)
target_test = np.asarray(target_test)

['Chen Aijun', 'Chen Jiayin', 'Dong Yini', 'Dong Zhiyong', 'Gao Yiqing', 'Jiang Letian', 'Li Baoyan', 'Li Xiufang', 'Liu Jianhua', 'Liu Mingzhen', 'Lv Hui', 'Wu Di', 'Wu Leru', 'Yu Bin', 'Yu Di', 'Yu Hong', 'Yu Shaozhou', 'Yu Wei', 'Yu Ying', 'Zheng Ruojun']


In [6]:
#图像数据转换特征矩阵
image_data_test = []

for image_test in images_test:
    #转换成一维的数组
    data_test = image_test.flatten()
    image_data_test.append(data_test)

#print(image_data)

In [7]:
#转换为numpy数组
X_test = np.array(image_data_test)
y_test = target_test
print(type(X_test))
print(X_test.shape)

<class 'numpy.ndarray'>
(600, 10000)


# 对训练集进行预处理

In [8]:
# 对数据进行中心化，即每一列减去该列的均值
X_train_centered = X_train - np.mean(X_train, axis=0)

In [18]:
# 主成分数量
k = 100
# 对训练矩阵进行SVD
U, S, Vt = np.linalg.svd(X_train_centered, full_matrices=False)
# 选择前100个主成分
X_train_pca = U[:, :k]
# 构建变换矩阵P
P = np.dot(X_train.T, X_train_pca)
print(Vt.shape)
print(P.shape)

(200, 10000)
(10000, 100)


In [10]:
# 计算平均向量

# 将向量切分成每30个一组
group_size = 10
num_groups = 20
grouped_vectors = X_train_pca.reshape(num_groups, group_size, -1)

# 计算每组的平均向量
X_train_average = np.mean(grouped_vectors, axis=1)

print(X_train_average.shape)

(20, 100)


# NCC进行人脸识别

1、用numpy函数

In [11]:
true_count = 0

In [12]:
# 计算每个测试数据和训练数据之间的欧几里得范数
for i in range(len(X_test)):
    test_vector = X_test[i]
    
    # 归一化测试向量
    test_vector = test_vector - np.mean(test_vector, axis=0)

    # 对测试向量PCA
    test_vector = np.dot(test_vector,P)
    
    # 初始化一个列表来存储每个训练向量与当前测试向量的欧几里得范数
    distances = []
    
    # 计算每个训练向量与当前测试向量的欧几里得范数
    for j in range(len(X_train_average)):
        train_vector = X_train_average[j]
        
        # 计算欧几里得范数并添加到列表中
        distance = np.linalg.norm(train_vector - test_vector)
        distances.append(distance)
        
    # 找到最小距离的训练向量索引
    min_distances_index = np.argmin(distances)
    
    # 检查索引是否相同
    if min_distances_index + 1 == (i // 30) +1:
        true_count += 1
        print(i)

10
11
12
13
15
16
17
18
19
20
21
22
23
24
25
26
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
213
214
216
217
218
219
221
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303

In [13]:
print(f"True的数量：{true_count}")
print(f"True的概率：{true_count/(len(X_test))}")

True的数量：526
True的概率：0.8766666666666667
